In [1]:
import pandas as pd
import numpy as np
import microdf as mdf
import plotly.express as px
# import ubicenter

In [2]:
p = pd.read_csv("data/nys_cps.csv.gz")
p.columns = p.columns.str.lower()
p.inctot.replace({999999999: 0}, inplace=True)
p.adjginc.replace({99999999: 0}, inplace=True)
# p[["asecwt", "spmwt"]] /= 100

In [3]:
NYC_COUNTY = pd.DataFrame(columns=["fips", "county", "pop_m"])
NYS_FIPS = 36
# Per https://en.wikipedia.org/wiki/List_of_counties_in_New_York
NYC_COUNTY.loc["Manhattan"] = pd.Series({"fips": 61, "county": "New York", "pop_m": 1.632})
NYC_COUNTY.loc["Brooklyn"] = pd.Series({"fips": 47, "county": "Kings", "pop_m": 2.59})
NYC_COUNTY.loc["The Bronx"] = pd.Series({"fips": 5, "county": "Bronx", "pop_m": 1.435})
NYC_COUNTY.loc["Staten Island"] = pd.Series({"fips": 85, "county": "Richmond", "pop_m": 0.475})
NYC_COUNTY.loc["Queens"] = pd.Series({"fips": 81, "county": "Queens", "pop_m": 2.287})
NYC_COUNTY["full_fips"] = NYS_FIPS * 1000 + NYC_COUNTY.fips

pn = p[p.county.isin(NYC_COUNTY.full_fips)][
    ["county", "asecwt", 'age', 'sex', 'race', 'hispan', 'inctot', 'spmwt', 'spmtotres', 'spmthresh',
       'spmfamunit', 'adjginc', 'ftotval', 'spmftotval']].copy()
NYC_COUNTY["asec_pop_m"] = NYC_COUNTY.full_fips.apply(lambda x: pn[pn.county == x].asecwt.sum() / 1e6)
NYC_COUNTY

,fips,county,pop_m,full_fips,asec_pop_m
Manhattan,61,New York,1.632,36061,1.634743
Brooklyn,47,Kings,2.590,36047,2.634859
The Bronx,5,Bronx,1.435,36005,1.586280
Staten Island,85,Richmond,0.475,36085,0.497162
Queens,81,Queens,2.287,36081,1.869408


In [4]:
# Add cumulative adult population to identify bottom 500,000 adults.
pna = pn[pn.age >= 18].copy()
pna.sort_values("spmftotval", inplace=True)
pna["pop_cum"] = pna.asecwt.cumsum()

spmtotres_thresh = pna[pna.pop_cum < 500000].spmtotres.max()

pna[pna.pop_cum < 500000]

,county,asecwt,age,sex,race,hispan,inctot,spmwt,spmtotres,spmthresh,spmfamunit,adjginc,ftotval,spmftotval,pop_cum
2587,36047,2344.91,76,2,200,0,0,2344.91,0.0,33380.0,8485001,0,0,0,2344.91
3459,36061,1986.83,37,1,100,0,0,1986.83,17876.0,30830.0,9057001,0,0,0,4331.74
3458,36061,1986.83,37,2,100,0,0,1986.83,17876.0,30830.0,9057001,0,0,0,6318.57
1149,36005,5823.38,67,1,200,400,0,5823.38,0.0,16240.0,7668003,0,0,0,12141.95
3365,36061,2272.33,62,2,200,0,0,2272.33,-300.0,22890.0,8940001,0,0,0,14414.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4122,36081,4959.12,80,1,100,0,12000,4959.12,11858.0,13640.0,9399001,0,12000,12000,483357.89
1844,36047,3225.03,55,1,651,0,12000,3225.03,22748.0,39020.0,8059001,12000,12000,12000,486582.92
3075,36061,4949.22,72,1,100,0,12000,4949.22,18504.0,16240.0,8726001,0,12000,12000,491532.14
1152,36005,3090.58,32,1,200,0,12000,3090.58,30979.0,29080.0,7673001,11152,12000,12000,494622.72


In [5]:
sn = pn[["spmfamunit", "spmwt", "spmftotval", "spmtotres", "spmthresh"]].drop_duplicates().sort_values("spmftotval")
sn["spmwt_cum"] = sn.spmwt.cumsum()
spmftotval_thresh = sn[sn.spmwt_cum < 500000].spmftotval.max()
sn[sn.spmftotval <= spmftotval_thresh].spmwt.sum()
sn["ubi"] = np.where(sn.spmftotval <= spmftotval_thresh, 2000, 0)
sn["spmtotres_ubi"] = sn.spmtotres + sn.ubi

In [6]:
mdf.weighted_sum(sn, "ubi", "spmwt")

1004590420.0

In [7]:
pn = pn.merge(sn[["spmfamunit", "spmtotres_ubi"]], on="spmfamunit")


In [8]:
mdf.poverty_gap(sn, "spmtotres_ubi", "spmthresh", "spmwt") / 1e9

5.53524380132

In [9]:
mdf.poverty_gap(sn, "spmtotres", "spmthresh", "spmwt") / 1e9

6.256897865129999

In [10]:
mdf.deep_poverty_rate(pn, "spmtotres", "spmthresh", "asecwt")

0.04859213974110767

In [11]:
mdf.deep_poverty_rate(pn, "spmtotres_ubi", "spmthresh", "asecwt")

0.04356069904096139

In [12]:
mdf.poverty_rate(pn, "spmtotres_ubi", "spmthresh", "asecwt")

0.18354143352276175

In [13]:
mdf.poverty_rate(pn, "spmtotres", "spmthresh", "asecwt")

0.19196885996563795

In [14]:
def pct_chg(base, reform):
    return (reform - base) / base

pct_chg(mdf.poverty_rate(pn, "spmtotres", "spmthresh", "asecwt"),
mdf.poverty_rate(pn, "spmtotres_ubi", "spmthresh", "asecwt"))

-0.04389996609025385

In [15]:
pct_chg(mdf.deep_poverty_rate(pn, "spmtotres", "spmthresh", "asecwt"),
mdf.deep_poverty_rate(pn, "spmtotres_ubi", "spmthresh", "asecwt"))

-0.10354433303314309

In [16]:
pct_chg(mdf.gini(sn, "spmtotres", "spmwt"),
mdf.gini(sn, "spmtotres_ubi", "spmwt"))

-0.01087395327199434

In [17]:
# NYS population: 19.45M official as of 2019.
p[p.statefip == NYS_FIPS].asecwt.sum() / 1e6

19.103431190000002

In [18]:
p[(p.statefip == NYS_FIPS) & p.county.isin(NYC_FIPS)].asecwt.sum() / 1e6

NameError: name 'NYC_FIPS' is not defined

In [22]:
p[p.county.isin([NYS_FIPS * 1000 + i for i in NYC_FIPS])].groupby("county").asecwt.sum()

county
36005    1586280.10
36047    2634859.16
36061    1634742.52
36081    1869407.84
36085     497162.45
Name: asecwt, dtype: float64

In [20]:
p[p.county == 36061].asecwt.sum()

1634742.52